# papeles package - institutions network analysis example

In this notebook, all institutions from Neurips papers are extracted and then an institutions network is created using the co-location of such institutions in the research papers. Data was obtained using the [neurips_crawler](https://github.com/glhuilli/neurips_crawler).

The `papeles` package is used to extract a clean version of the institutions by processing the front page of the research papers, identifying from there which institutions were involved in the research. Results shows that...



In [1]:
import os
import json
from collections import defaultdict

from tqdm.notebook import tqdm


from papeles.paper.neurips import get_key


## Loading the data

Data is already available in `/var/data/neurips_analysis`. To run this script make sure to create this folder and download the data from github repository `xxx`.

These are files already processed from `pdf` to `txt`, and then only selecting the `header` of such papers (everything from the beginning of the document until the abstract) using the `papeles` package. For more details on this, please check the ipython notebook `xxx`. Also, there are two files with keywords already extracted from the entire corpus, also using the `papeles` package. For more details on this, please check the ipython notebook `xxx`

In [7]:
# These are files with encoding issues that were not parse correctly by the pdf_parser 
SKIP_FILES = [
    '5049-nonparametric-multi-group-membership-model-for-dynamic-networks.pdf_headers.txt',
    '4984-cluster-trees-on-manifolds.pdf_headers.txt',
    '5820-alternating-minimization-for-regression-problems-with-vector-valued-outputs.pdf_headers.txt',
    '9065-visualizing-and-measuring-the-geometry-of-bert.pdf_headers.txt'
    '4130-implicit-encoding-of-prior-probabilities-in-optimal-neural-populations.pdf_headers.txt',
    '7118-local-aggregative-games.pdf_headers.txt'
]

NEURIPS_ANALYSIS_DATA_PATH = '/var/data/neurips_analysis'

file_lines = defaultdict(list)
for filename in tqdm(os.listdir(os.path.join(NEURIPS_ANALYSIS_DATA_PATH, 'files_headers/')), 'loading files'):
    if filename in SKIP_FILES:
        continue
    with open(os.path.join(NEURIPS_ANALYSIS_DATA_PATH, './files_headers/', filename), 'r') as f:
        for line in f.readlines():
            file_lines[get_key(filename)].append(line.strip())
            
metadata_path = os.path.join(NEURIPS_ANALYSIS_DATA_PATH, 'files_metadata/')

metadata = {}
for filename in tqdm(os.listdir(metadata_path), 'loading metadata'):
    with open(os.path.join(metadata_path, filename), 'r') as f: # open in readonly mode
        for line in f.readlines():
            data = json.loads(line)
            metadata[get_key(data['pdf_name'])] = data


with open(os.path.join(NEURIPS_ANALYSIS_DATA_PATH, 'year_keywords_counter_n2.json'), 'r') as f:
    keywords_n2 = json.load(f)
    

with open(os.path.join(NEURIPS_ANALYSIS_DATA_PATH, 'year_keywords_counter_n3.json'), 'r') as f:
    keywords_n3 = json.load(f)

print(f'keywords loaded -- 2-grams: {sum([len(v) for k, v in keywords_n2.items()])}, 3-grams: {sum([len(v) for k, v in keywords_n3.items()])}')



keywords loaded -- 2-grams: 118478, 3-grams: 120341


In [3]:
from papeles.paper.neurips import institutions

from collections import Counter 
import itertools


In [8]:
inst_counter = Counter()
for file, lines in list(file_lines.items()):
    file_institutions = institutions.get_file_institutions(lines)
    unique_file_institutions = list(set(file_institutions))
    inst_counter.update(unique_file_institutions)

cleaned = len(sorted([x for x in inst_counter.items() if x[1] > 0 and x[0]], key= lambda x: x[1], reverse=True))
total = sum([x[1] for x in inst_counter.items() if x[1] > 0 and x[0]])

print(f'current institutions: {cleaned}')
print(f'total raw institutions: {total}')
print(f'clean-up fraction: {"{:0.2f}".format(1 - cleaned / total)}')

current institutions: 2989
total raw institutions: 9245
clean-up fraction: 0.68


## Institutions Interactions Graph 

To build the graph, we'll use both `networkx` and the `papeles` package. In particular, we'll use a method tailored for Neurips data called `institutions_graph`. I recommend looking into the details of this method, but the overall goal is that for each file of a particular `year` (optional) in the corpus (`file_lines`), it will try to find all institutions (from `inst_counter`) that co-authored that particular file. This will only consider institutions with at least a minimum frequency (`freq`) from `inst_counter`. 

I'm also importing the `dump_to_d3js` method which will be useful to generate the output needed for a second part of this analysis. 



In [9]:
import networkx as nx

from papeles.paper.neurips import institutions_graph, dump_to_d3js

In [9]:
g_all_n2, g_all_n2_files = institutions_graph.build_institutions_graph(file_lines, metadata, inst_counter, freq=2) 
print(nx.info(g_all_n2))


Name: 
Type: Graph
Number of nodes: 312
Number of edges: 1828
Average degree:  11.7179


This means that the graph with considering all institutions that appear at least twice in all Neurips papers since 2009 is 312, and it's highgly connected (avg degree 11.71). I'll get back to the graph analysis later, but let's see how this graph looks like if we consider institutions with at least 5 papers in Neurips. 


In [11]:
g_all_n5, g_all_n5_files = institutions_graph.build_institutions_graph(file_lines, metadata, inst_counter, freq=5) 
print(nx.info(g_all_n5))

Name: 
Type: Graph
Number of nodes: 172
Number of edges: 1392
Average degree:  16.1860


In [25]:
dump_to_d3js(g_all_n5, os.path.join(NEURIPS_ANALYSIS_DATA_PATH, 'n5_graph_all_years.json'))


The graph is now almost ~50% smaller (in terms of nodes), but the number of edges didn't change much (reduced only a ~70%). Something we can conclude from this, is that ~50% of the nodes in the graph didn't contribute many connections (which is expected as they had less papers). However, the degree increased by +45%, which again tells us that it's a much richer graph to analyze. 

If we consider the graphs based on the papers associated to every year since 2009 to 2019, at the same frequency (freq=5), it's possible to see how Neurips took a serious turn around year ~2013 when it became clearly well known and more researchers started publishing at Neurips. 

In [12]:
graphs = {}
graph_files = {}
for year in range(2009, 2020):
    graphs[year], graph_files[year] = institutions_graph.build_institutions_graph(file_lines, metadata, inst_counter, freq=5, year=year)
    print(f'\nyear: {year}')
    print(nx.info(graphs[year]))



year: 2009
Name: 
Type: Graph
Number of nodes: 81
Number of edges: 127
Average degree:   3.1358

year: 2010
Name: 
Type: Graph
Number of nodes: 84
Number of edges: 146
Average degree:   3.4762

year: 2011
Name: 
Type: Graph
Number of nodes: 79
Number of edges: 150
Average degree:   3.7975

year: 2012
Name: 
Type: Graph
Number of nodes: 95
Number of edges: 174
Average degree:   3.6632

year: 2013
Name: 
Type: Graph
Number of nodes: 99
Number of edges: 164
Average degree:   3.3131

year: 2014
Name: 
Type: Graph
Number of nodes: 104
Number of edges: 192
Average degree:   3.6923

year: 2015
Name: 
Type: Graph
Number of nodes: 113
Number of edges: 238
Average degree:   4.2124

year: 2016
Name: 
Type: Graph
Number of nodes: 120
Number of edges: 271
Average degree:   4.5167

year: 2017
Name: 
Type: Graph
Number of nodes: 135
Number of edges: 349
Average degree:   5.1704

year: 2018
Name: 
Type: Graph
Number of nodes: 141
Number of edges: 405
Average degree:   5.7447

year: 2019
Name: 
Type: 

In [23]:
for k, d in sorted(year_keys.items(), key=lambda x: x[0]):
    print(k, len(d))

2009 261
2010 292
2011 305
2012 368
2013 360
2014 411
2015 403
2016 569
2017 679
2018 1008
2019 1427


In [151]:
plot_graph(g_all)

In [149]:
for year in range(2009, 2020):
    print(f'\nyear: {year}')
    print(nx.info(graphs[year]))
    partition_y = community.best_partition(graphs[year])
    institution_clusters_y = defaultdict(list)
    for k, p in partition_y.items():
        institution_clusters_y[p].append(k)
    pprint(institution_clusters_y)
    


year: 2009
Name: 
Type: Graph
Number of nodes: 67
Number of edges: 110
Average degree:   3.2836
defaultdict(<class 'list'>,
            {0: ['universite de montreal'],
             1: ['stanford university',
                 'university of california berkeley',
                 'brown university',
                 'university of texas at austin'],
             2: ['boston university'],
             3: ['university of copenhagen'],
             4: ['university of alberta',
                 'indian institute of science',
                 'mcgill university'],
             5: ['duke university',
                 'princeton university',
                 'university of maryland'],
             6: ['university of oxford',
                 'carnegie mellon university',
                 'university of toronto',
                 'mit',
                 'max planck institute',
                 'inria',
                 'intel labs',
                 'saarland university'],
             7: ['goo

In [146]:
dendogram = community.generate_dendrogram(g_all)
print(dendogram)
partition_1 = community.partition_at_level(dendogram, 0)
institution_clusters_1 = defaultdict(list)
for k, p in partition_1.items():    
    institution_clusters_1[p].append(k)
institution_clusters_1


[{'university of oxford': 0, 'cornell university': 1, 'australian national university': 0, 'university of alberta': 2, 'carnegie mellon university': 3, 'deepmind': 4, 'university of washington': 3, 'northeastern university': 5, 'columbia university': 6, 'stanford university': 3, 'university of amsterdam': 7, 'google research': 7, 'university of california irvine': 8, 'microsoft research': 3, 'university of massachusetts amherst': 3, 'hong kong university of science and technology': 9, 'inria': 10, 'university of copenhagen': 10, 'university of toronto': 11, 'universite de montreal': 4, 'university of cambridge': 0, 'university of california berkeley': 3, 'princeton university': 6, 'google brain': 4, 'university of edinburgh': 0, 'mcgill university': 2, 'university of waterloo': 5, 'university of british columbia': 4, 'university of texas at austin': 3, 'university of pennsylvania': 3, 'technion - israel institute of technology': 3, 'university of california los angeles': 0, 'new york u

defaultdict(list,
            {0: ['university of oxford',
              'australian national university',
              'university of cambridge',
              'university of edinburgh',
              'university of california los angeles',
              'alan turing institute',
              'max planck institute for intelligent systems',
              'university college london',
              'imperial college london',
              'university of sydney',
              'technische universitat berlin',
              'university of warwick'],
             1: ['cornell university', 'rutgers university'],
             2: ['university of alberta',
              'mcgill university',
              'indian institute of science'],
             3: ['carnegie mellon university',
              'university of washington',
              'stanford university',
              'microsoft research',
              'university of massachusetts amherst',
              'university of california berkele

In [31]:
partition = community.best_partition(g_all)
institution_clusters = defaultdict(list)
for k, p in partition.items():
    institution_clusters[p].append(k)
institution_clusters
    

defaultdict(list,
            {0: ['university of oxford',
              'australian national university',
              'deepmind',
              'universite de montreal',
              'university of cambridge',
              'google brain',
              'university of edinburgh',
              'university of british columbia',
              'alan turing institute',
              'max planck institute for intelligent systems',
              'university college london',
              'rensselaer polytechnic institute',
              'uber',
              'facebook ai research',
              'imperial college london',
              'university of sydney',
              'technische universitat berlin',
              'university of warwick'],
             1: ['cornell university',
              'columbia university',
              'hong kong university of science and technology',
              'princeton university',
              'university of pennsylvania',
              'purdue uni

In [33]:
partition = community.best_partition(g_all_n2)
institution_clusters = defaultdict(list)
for k, p in partition.items():
    institution_clusters[p].append(k)
institution_clusters
    

defaultdict(list,
            {0: ['university of oxford',
              'university of cambridge',
              'university of edinburgh',
              'university of california los angeles',
              'alan turing institute',
              'max planck institute for intelligent systems',
              'city university of hong kong',
              'university college london',
              'korea university',
              'university of bristol',
              'university of virginia',
              'nanyang technological university',
              'imperial college london',
              'national university of defense technology',
              'ku leuven',
              'technische universitat berlin',
              'university of southampton',
              'king’s college london',
              'universidad de chile',
              'university carlos iii in madrid',
              'lancaster university',
              'university of edinburgh uk',
              'eurecom',
  

In [34]:
nx.triangles(g_all_n2)

{'university of oxford': 257,
 'cornell university': 152,
 'australian national university': 7,
 'nicta': 5,
 'university of alberta': 44,
 'iowa state university': 3,
 'carnegie mellon university': 609,
 'deepmind': 199,
 'university of washington': 242,
 'northeastern university': 66,
 'columbia university': 284,
 'stanford university': 457,
 'university of pittsburgh': 2,
 'university of amsterdam': 7,
 'google research': 406,
 'university of california irvine': 23,
 'microsoft research': 642,
 'university of massachusetts amherst': 19,
 'hong kong university of science and technology': 0,
 'inria': 136,
 'ecole centrale paris': 1,
 'university of copenhagen': 15,
 'university of toronto': 165,
 'vmware research': 0,
 'universite de montreal': 41,
 'university of cambridge': 173,
 'university of california berkeley': 641,
 'princeton university': 444,
 'google brain': 306,
 'university of edinburgh': 67,
 'peng cheng laboratory': 0,
 'mcgill university': 6,
 'university of waterloo'

In [38]:
all_cliques = nx.enumerate_all_cliques(g_all_n2)
triad_cliques = [x for x in all_cliques if len(x)==3 ]
len(triad_cliques)

4275

In [136]:
hubs = [('microsoft research', 0.0536),
 ('carnegie mellon university', 0.0413),
 ('mit', 0.0404),
 ('university of california berkeley', 0.0402),
 ('princeton university', 0.0356),
 ('university of texas at austin', 0.0303),
 ('stanford university', 0.0289),
 ('columbia university', 0.0261),
 ('google research', 0.0242),
 ('harvard university', 0.0233),
 ('google', 0.0197),
 ('deepmind', 0.0197),
 ('university of pennsylvania', 0.0188),
 ('university of washington', 0.0183),
 ('google brain', 0.0169),
 ('university of toronto', 0.0168),
 ('university of california san diego', 0.0164),
 ('university of cambridge', 0.0161),
 ('new york university', 0.0157),
 ('university of oxford', 0.0144),
 ('cornell university', 0.0132),
]
hubs_nodes = [x[0] for x in hubs]
data_for_teams = defaultdict(list)
for node in g_all_n2.nodes():
    if node in hubs_nodes:
        continue
    for edge in g_all_n2.edges(node):
        if g_all_n2[edge[0]][edge[1]]['weight'] < 3:
            continue
        for e in edge:
            if e != node and e not in hubs_nodes:
                data_for_teams[node].append(e)

for k, v in data_for_teams.items():
    print(k, len(v))

australian national university 1
nicta 1
university of alberta 1
university of california irvine 1
inria 3
university of edinburgh 1
technion - israel institute of technology 1
university of california los angeles 3
toyota technological institute at chicago 1
boston university 1
university of illinois at urbana-champaign 2
purdue university 1
indian institute of science 1
alan turing institute 2
chinese university of hong kong 1
ibm research 4
georgia institute of technology 2
university college london 3
rutgers university 2
university of michigan 3
university of wisconsin-madison 1
duke university 2
eth zürich 3
riken aip 4
nagoya institute of technology 1
johns hopkins university 1
ecole normale superieure 1
national university of singapore 1
university of minnesota 3
dartmouth college 1
college park 1
epfl 3
yale university 1
nec corporation 5
university of tokyo 2
university of virginia 1
university of tübingen 1
max planck institute 3
imperial college london 2
national university 

In [134]:
data_for_teams#['georgia institute of technology']

defaultdict(list,
            {'australian national university': ['nicta'],
             'nicta': ['australian national university'],
             'inria': ['ecole normale superieure', 'cnrs'],
             'toyota technological institute at chicago': ['university of chicago'],
             'ibm research': ['georgia institute of technology'],
             'georgia institute of technology': ['ibm research'],
             'eth zürich': ['epfl'],
             'ecole normale superieure': ['inria'],
             'epfl': ['eth zürich'],
             'university of tokyo': ['tokyo institute of technology'],
             'university of chicago': ['toyota technological institute at chicago'],
             'cnrs': ['inria'],
             'rice university': ['baylor college of medicine'],
             'baylor college of medicine': ['rice university'],
             'tokyo institute of technology': ['nikon corporation',
              'university of tokyo'],
             'nikon corporation': ['tokyo

In [109]:
from pprint import pprint

pprint(data_for_teams)

defaultdict(<class 'list'>,
            {'aalto university': ['university of helsinki'],
             'alan turing institute': ['university of california los angeles',
                                       'university college london'],
             'alibaba group': ['georgia institute of technology'],
             'australian national university': ['nicta',
                                                'university of sydney'],
             'baidu research usa': ['rutgers university'],
             'baylor college of medicine': ['rice university'],
             'boston university': ['ibm research'],
             'chalmers university of technology': ['indian institute of '
                                                   'science'],
             'champalimaud centre for the unknown': ['ecole normale '
                                                     'superieure'],
             'chinese university of hong kong': ['johns hopkins university',
                                       

In [125]:
"""
1 -> [2, 3]

2 -> [1, 3, 4]

5 -> [6]

[1, 2, 3, 4]

[5, 6]
"""

def get_teams(dft):
    teams = []
    sets = [set(v).union({k}) for k, v in dft.items()]
    print(len(sets))
    j = 1
    added = {0}
    for i in range(0, len(sets)):
        print(f'added: {added}')
        if i in added:
            continue
        team = set()
        team = team.union(sets[i])
        if 'alibaba group' in team:
            print(f'(i, j) {i, j} (i)')
        for j in range(i+1, len(sets)):
            if j in added:
                continue

            if 'alibaba group' in team:
                print(f'(i, j) {i, j} (j)')
            if sets[j].intersection(team):
                if 'alibaba group' in sets[j]:
                    print(f'adding in j {j}')
                team = team.union(sets[j])
                added.add(j)
        teams.append(team)
    return teams
  
test = {1: [2, 3], 2: [1, 3, 4, 5], 3: [2], 4: [2], 5: [6, 2], 6: [5]}
get_teams(test)
# len(teams)
    
#     for k2, v2 in data_for_teams.items():
#         if k == k2:
#             continue
#         if k in v2:
            

6
added: {0}
added: {0}
added: {0, 2, 3, 4, 5}
added: {0, 2, 3, 4, 5}
added: {0, 2, 3, 4, 5}
added: {0, 2, 3, 4, 5}


[{1, 2, 3, 4, 5, 6}]

In [129]:
teams = get_teams(data_for_teams)


120
added: {0}
added: {0}
adding in j 28
(i, j) (1, 29) (j)
(i, j) (1, 30) (j)
(i, j) (1, 31) (j)
(i, j) (1, 32) (j)
(i, j) (1, 33) (j)
(i, j) (1, 34) (j)
(i, j) (1, 35) (j)
(i, j) (1, 36) (j)
(i, j) (1, 37) (j)
(i, j) (1, 38) (j)
(i, j) (1, 39) (j)
(i, j) (1, 40) (j)
(i, j) (1, 41) (j)
(i, j) (1, 42) (j)
(i, j) (1, 43) (j)
(i, j) (1, 44) (j)
(i, j) (1, 45) (j)
(i, j) (1, 46) (j)
(i, j) (1, 47) (j)
(i, j) (1, 48) (j)
(i, j) (1, 49) (j)
(i, j) (1, 50) (j)
(i, j) (1, 51) (j)
(i, j) (1, 52) (j)
(i, j) (1, 53) (j)
(i, j) (1, 54) (j)
(i, j) (1, 55) (j)
(i, j) (1, 56) (j)
(i, j) (1, 57) (j)
(i, j) (1, 58) (j)
(i, j) (1, 59) (j)
(i, j) (1, 60) (j)
(i, j) (1, 61) (j)
(i, j) (1, 62) (j)
(i, j) (1, 63) (j)
(i, j) (1, 64) (j)
(i, j) (1, 65) (j)
(i, j) (1, 66) (j)
(i, j) (1, 67) (j)
(i, j) (1, 68) (j)
(i, j) (1, 69) (j)
(i, j) (1, 70) (j)
(i, j) (1, 71) (j)
(i, j) (1, 72) (j)
(i, j) (1, 73) (j)
(i, j) (1, 74) (j)
(i, j) (1, 75) (j)
(i, j) (1, 76) (j)
(i, j) (1, 77) (j)
(i, j) (1, 78) (j)
adding in

In [130]:
pprint(teams)

[{'alan turing institute',
  'alibaba group',
  'australian national university',
  'baidu research usa',
  'boston university',
  'chalmers university of technology',
  'champalimaud centre for the unknown',
  'chinese university of hong kong',
  'cnrs',
  'college park',
  'deepmind',
  'duke university',
  'ecole normale superieure',
  'ecole polytechnique',
  'epfl',
  'eth zürich',
  'facebook ai research',
  'georgia institute of technology',
  'ibm research',
  'idiap research institute',
  'imperial college london',
  'indian institute of science',
  'indiana university',
  'inria',
  'institute for advanced study',
  'irisa',
  'ist austria',
  'johns hopkins university',
  'jst presto',
  'keio university',
  'korea university',
  'kth',
  'lcsl – iit & mit',
  'machine learning research group',
  'max planck institute',
  'max planck institute for intelligent systems',
  'nagoya institute of technology',
  'national institute of informatics',
  'national university of defens

In [131]:
for t in teams:
    if 'alibaba group' in t:
        print(len(t))

89


In [54]:
# https://stackoverflow.com/questions/10301000/python-connected-components

myGraph = {0: [1,2,3], 1: [], 2: [1], 3: [4,5],4: [3,5], 5: [3,4,7], 6: [8], 7: [],8: [9], 9: []}


def getRoots(aNeigh):
    def findRoot(aNode,aRoot):
        while aNode != aRoot[aNode][0]:
            aNode = aRoot[aNode][0]
        return (aNode,aRoot[aNode][1])
    myRoot = {} 
    for myNode in aNeigh.keys():
        myRoot[myNode] = (myNode,0)  
    for myI in aNeigh: 
        for myJ in aNeigh[myI]: 
            (myRoot_myI,myDepthMyI) = findRoot(myI,myRoot) 
            (myRoot_myJ,myDepthMyJ) = findRoot(myJ,myRoot) 
            if myRoot_myI != myRoot_myJ: 
                myMin = myRoot_myI
                myMax = myRoot_myJ 
                if  myDepthMyI > myDepthMyJ: 
                    myMin = myRoot_myJ
                    myMax = myRoot_myI
                myRoot[myMax] = (myMax,max(myRoot[myMin][1]+1,myRoot[myMax][1]))
                myRoot[myMin] = (myRoot[myMax][0],-1) 
    myToRet = {}
    for myI in aNeigh: 
        if myRoot[myI][0] == myI:
            myToRet[myI] = []
    for myI in aNeigh: 
        myToRet[findRoot(myI,myRoot)[0]].append(myI) 
    return myToRet 


getRoots(graph_for_teams).keys()

dict_keys(['university of edinburgh', 'radboud university nijmegen', 'idsia', 'university of helsinki', 'iit delhi'])

In [38]:
centrality = nx.eigenvector_centrality(graph)
sorted([(v, float('{:0.4f}'.format(c))) for v, c in centrality.items()], key=lambda x: x[1], reverse=True)


[('microsoft research', 0.2432),
 ('carnegie mellon university', 0.2383),
 ('university of california berkeley', 0.2298),
 ('mit', 0.2287),
 ('princeton university', 0.1906),
 ('stanford university', 0.1891),
 ('google', 0.1862),
 ('google research', 0.175),
 ('harvard university', 0.1622),
 ('university of pennsylvania', 0.1609),
 ('columbia university', 0.1578),
 ('university of texas at austin', 0.1532),
 ('google brain', 0.1446),
 ('university of washington', 0.1427),
 ('ibm research', 0.1407),
 ('university of oxford', 0.1391),
 ('new york university', 0.1383),
 ('university of california san diego', 0.137),
 ('university college london', 0.1298),
 ('university of michigan', 0.1271),
 ('deepmind', 0.1245),
 ('duke university', 0.1235),
 ('tsinghua university', 0.12),
 ('university of cambridge', 0.1193),
 ('university of california los angeles', 0.1158),
 ('toyota technological institute at chicago', 0.1144),
 ('university of illinois at urbana-champaign', 0.114),
 ('cornell unive

In [41]:
centrality = nx.katz_centrality_numpy(graph)
sorted([(v, float('{:0.4f}'.format(c))) for v, c in centrality.items()], key=lambda x: x[1], reverse=True)


[('inria', 0.2136),
 ('ecole normale superieure', 0.1634),
 ('university of oxford', 0.1406),
 ('eth zürich', 0.1383),
 ('university of cambridge', 0.135),
 ('cnrs', 0.131),
 ('max planck institute for intelligent systems', 0.1309),
 ('deepmind', 0.1248),
 ('facebook ai research', 0.1083),
 ('university of copenhagen', 0.1056),
 ('alan turing institute', 0.1052),
 ('university college london', 0.101),
 ('sorbonne universite', 0.0932),
 ('max planck institute', 0.0928),
 ('ecole polytechnique', 0.0883),
 ('universitat pompeu fabra', 0.0871),
 ('imperial college london', 0.0855),
 ('new york university', 0.0807),
 ('university of edinburgh', 0.0771),
 ('epfl', 0.0768),
 ('google brain', 0.0767),
 ('university of sydney', 0.0765),
 ('mit', 0.0733),
 ('ist austria', 0.0706),
 ('uber', 0.0704),
 ('university of genoa', 0.0696),
 ('leiden university', 0.0674),
 ('australian national university', 0.0654),
 ('jst presto', 0.0653),
 ('university of alberta', 0.0649),
 ('university of warwick', 

In [42]:
centrality = nx.closeness_centrality(graph)
sorted([(v, float('{:0.4f}'.format(c))) for v, c in centrality.items()], key=lambda x: x[1], reverse=True)


[('microsoft research', 0.5165),
 ('carnegie mellon university', 0.5107),
 ('mit', 0.5107),
 ('university of california berkeley', 0.5071),
 ('stanford university', 0.4911),
 ('princeton university', 0.4743),
 ('google research', 0.4706),
 ('google', 0.4663),
 ('university of oxford', 0.4645),
 ('harvard university', 0.461),
 ('university college london', 0.4604),
 ('columbia university', 0.4574),
 ('new york university', 0.4551),
 ('university of pennsylvania', 0.45),
 ('ibm research', 0.45),
 ('university of cambridge', 0.4489),
 ('deepmind', 0.4483),
 ('google brain', 0.4483),
 ('university of california san diego', 0.4478),
 ('university of washington', 0.4467),
 ('university of texas at austin', 0.4461),
 ('duke university', 0.4434),
 ('inria', 0.4428),
 ('cornell university', 0.439),
 ('university of california los angeles', 0.439),
 ('eth zürich', 0.4364),
 ('university of illinois at urbana-champaign', 0.4348),
 ('tsinghua university', 0.4348),
 ('university of michigan', 0.433

In [43]:
centrality = nx.information_centrality(graph)
sorted([(v, float('{:0.4f}'.format(c))) for v, c in centrality.items()], key=lambda x: x[1], reverse=True)

NetworkXError: Graph not connected.

In [44]:
centrality = nx.betweenness_centrality(graph)
sorted([(v, float('{:0.4f}'.format(c))) for v, c in centrality.items()], key=lambda x: x[1], reverse=True)


[('mit', 0.1453),
 ('microsoft research', 0.1286),
 ('university of california berkeley', 0.1044),
 ('carnegie mellon university', 0.1032),
 ('stanford university', 0.0765),
 ('university college london', 0.0638),
 ('university of oxford', 0.0513),
 ('google', 0.0511),
 ('inria', 0.0439),
 ('columbia university', 0.0382),
 ('university of texas at austin', 0.0363),
 ('princeton university', 0.0354),
 ('university of cambridge', 0.0351),
 ('google research', 0.0346),
 ('cnrs', 0.0336),
 ('university of california los angeles', 0.0333),
 ('new york university', 0.0318),
 ('cornell university', 0.0307),
 ('harvard university', 0.0291),
 ('duke university', 0.0274),
 ('deepmind', 0.026),
 ('ntt communication science laboratories', 0.0257),
 ('ibm research', 0.0246),
 ('university of pennsylvania', 0.0232),
 ('university of illinois at urbana-champaign', 0.0228),
 ('university of alberta', 0.0226),
 ('eth zürich', 0.0225),
 ('rutgers university', 0.0216),
 ('chinese university of hong kong'

In [45]:
centrality = nx.current_flow_betweenness_centrality(graph)
sorted([(v, float('{:0.4f}'.format(c))) for v, c in centrality.items()], key=lambda x: x[1], reverse=True)


NetworkXError: Graph not connected.

In [46]:
centrality = nx.communicability_betweenness_centrality(graph)
sorted([(v, float('{:0.4f}'.format(c))) for v, c in centrality.items()], key=lambda x: x[1], reverse=True)


/Users/gastonlhuillier/Personal/papeles/.python/papeles/lib/python3.8/site-packages/networkx/algorithms/centrality/subgraph_alg.py:246: RuntimeWarning: invalid value encountered in true_divide
  B = (expA - scipy.linalg.expm(A.A)) / expA


[('australian national university', nan),
 ('nicta', nan),
 ('university of alberta', nan),
 ('cifar', nan),
 ('university of amsterdam', nan),
 ('microsoft research', nan),
 ('university of massachusetts amherst', nan),
 ('ecole centrale paris', nan),
 ('inria', nan),
 ('cornell university', nan),
 ('university of california berkeley', nan),
 ('princeton university', nan),
 ('ebay inc', nan),
 ('stanford university', nan),
 ('allen institute for ai', nan),
 ('university of texas at austin', nan),
 ('university of illinois at urbana-champaign', nan),
 ('openai', nan),
 ('brown university', nan),
 ('bar-ilan university', nan),
 ('harvard university', nan),
 ('university of verona', nan),
 ('indian institute of science', nan),
 ('chalmers university of technology', nan),
 ('toyota technological institute at chicago', nan),
 ('max planck institute for intelligent systems', nan),
 ('alan turing institute', nan),
 ('sun yat-sen university', nan),
 ('chinese university of hong kong', nan),
 

In [47]:
hubs, authorities = nx.hits(graph)

In [48]:
sorted([(v, float('{:0.4f}'.format(c))) for v, c in hubs.items()], key=lambda x: x[1], reverse=True)

[('microsoft research', 0.0536),
 ('carnegie mellon university', 0.0413),
 ('mit', 0.0404),
 ('university of california berkeley', 0.0402),
 ('princeton university', 0.0356),
 ('university of texas at austin', 0.0303),
 ('stanford university', 0.0289),
 ('columbia university', 0.0261),
 ('google research', 0.0242),
 ('harvard university', 0.0233),
 ('google', 0.0197),
 ('university of pennsylvania', 0.0188),
 ('university of washington', 0.0183),
 ('google brain', 0.0169),
 ('university of toronto', 0.0168),
 ('university of california san diego', 0.0164),
 ('university of cambridge', 0.0161),
 ('new york university', 0.0157),
 ('university of oxford', 0.0144),
 ('cornell university', 0.0132),
 ('duke university', 0.0119),
 ('deepmind', 0.0119),
 ('ibm research', 0.0112),
 ('university college london', 0.0107),
 ('georgia institute of technology', 0.0105),
 ('tsinghua university', 0.0102),
 ('university of michigan', 0.0101),
 ('toyota technological institute at chicago', 0.0099),
 ('t

In [49]:
sorted([(v, float('{:0.4f}'.format(c))) for v, c in authorities.items()], key=lambda x: x[1], reverse=True)

[('microsoft research', 0.0536),
 ('carnegie mellon university', 0.0413),
 ('mit', 0.0404),
 ('university of california berkeley', 0.0402),
 ('princeton university', 0.0356),
 ('university of texas at austin', 0.0303),
 ('stanford university', 0.0289),
 ('columbia university', 0.0261),
 ('google research', 0.0242),
 ('harvard university', 0.0233),
 ('google', 0.0197),
 ('university of pennsylvania', 0.0188),
 ('university of washington', 0.0183),
 ('google brain', 0.0169),
 ('university of toronto', 0.0168),
 ('university of california san diego', 0.0164),
 ('university of cambridge', 0.0161),
 ('new york university', 0.0157),
 ('university of oxford', 0.0144),
 ('cornell university', 0.0132),
 ('duke university', 0.0119),
 ('deepmind', 0.0119),
 ('ibm research', 0.0112),
 ('university college london', 0.0107),
 ('georgia institute of technology', 0.0105),
 ('tsinghua university', 0.0102),
 ('university of michigan', 0.0101),
 ('toyota technological institute at chicago', 0.0099),
 ('t

In [ ]:
# [
#     {
#         "name":"flare.analytics.cluster.AgglomerativeCluster",
#         "size":3938,
#         "imports":[
#             "flare.animate.Transitioner",
#             "flare.vis.data.DataList",
#             "flare.util.math.IMatrix",
#             "flare.analytics.cluster.MergeEdge",
#             "flare.analytics.cluster.HierarchicalCluster",
#             "flare.vis.data.Data"
#         ]
#     },
# ]